In [11]:
import os
from pydap.client import open_url
import xarray as xr
import datetime
from datetime import timedelta, date
import netCDF4 as nc
from netCDF4 import Dataset
import os
import sys
sys.path.append(os.path.abspath(".."))
from utils.degree_day_equations import *
from utils.net_cdf_functions import *
from utils.processing_functions import *
#from utils.visualization_functions import *

import pandas as pd
#from visualization_functions import *
import numpy as np
data_path =  "../data/"

In [15]:
#turn bbox into a dict of lat and lon with lat_min long_max etc 36.405810,-119.855347,36.710265,-119.200287
def bbox_to_dict(bbox):
    bbox = bbox.split(",")
    bbox_dict = {}
    bbox_dict["lat_min"] = float(bbox[0])
    bbox_dict["lon_min"] = float(bbox[1])
    bbox_dict["lat_max"] = float(bbox[2])
    bbox_dict["lon_max"] = float(bbox[3])
    return bbox_dict

bbox = "36.405810,-119.855347,36.710265,-119.200287"
bbox_dict = bbox_to_dict(bbox)
LTT = 9.4
UTT = 999
'''
data = testing_load_data(data_path , "2000-01-01", lat = 36.558188, lon = -119.527130, LTT = 9.4, UTT = 999, window = 2 )
data.to_netcdf("../data/temp/orange_grvove_test.nc")
'''
#load data
combined_dd_data = xr.open_dataset("../data/temp/orange_grvove_test.nc")
mod_1_dd_data = combined_dd_data
mod_1_dd_data['day_of_year'] = mod_1_dd_data.indexes['t'].dayofyear

mod_1_dd_data = mod_1_dd_data.set_coords('day_of_year')
mod_1_data = mod_1_dd_data.groupby('day_of_year').mean('t')
mod_1_data_dd = mod_1_data['degree_days'].values


In [21]:
def fflies_model_run_wrapper(date, data, dd_threshold, use_current_years_weather):
    # Assuming data is a single point (1x1xn) xarray DataArray
    return fflies_model_run(date, data, dd_threshold, use_current_years_weather)

def apply_fflies_model_run( data, date, dd_threshold=754):
    # Apply the wrapper function over the x and y dimensions
    result = xr.apply_ufunc(
        fflies_model_1,
       
        data,
        date,
        dd_threshold,
        input_core_dims=[[], [], []],
        output_core_dims=[[]],
        vectorize=True,
        #dask='parallelized',
        output_dtypes=[]
    )
    return result


In [22]:
mod_1_data_dd

array([[[1.11724069, 1.12946998, 1.1418984 , ..., 0.02373434,
         0.02427396, 0.02724962],
        [1.11729331, 1.12142484, 1.12298107, ..., 0.03833613,
         0.0322252 , 0.03253062],
        [1.11392136, 1.11934388, 1.12574868, ..., 0.04860116,
         0.04603075, 0.04255697],
        ...,
        [       nan,        nan,        nan, ..., 1.23974897,
         1.23921547, 1.21319487],
        [       nan,        nan,        nan, ..., 1.18137672,
         1.21596719, 1.24809576],
        [       nan,        nan,        nan, ..., 1.14547317,
         1.16703667, 1.18173825]],

       [[1.12932948, 1.12775127, 1.13459264, ..., 0.07556808,
         0.07715857, 0.07306543],
        [1.13567055, 1.12297395, 1.11095491, ..., 0.07086146,
         0.08163194, 0.08871507],
        [1.10811132, 1.11145726, 1.11009404, ..., 0.07713605,
         0.08378671, 0.09321737],
        ...,
        [       nan,        nan,        nan, ..., 1.07147758,
         1.05578847, 1.10278838],
        [   

In [23]:
result = apply_fflies_model_run( mod_1_data_dd, 10, dd_threshold=754)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
import xarray as xr
from glob import glob

def read_netcdfs(files, coords, LTT, UTT):
    def process_one_path(path):
        # Use a context manager to ensure the file gets closed after use
        with xr.open_dataset(path) as ds:
            # Initialize an empty list to store data for each point or bounding box
            point_datasets = []
            
            if isinstance(coords, tuple) and len(coords) == 2:
                # Single point case
                lat, lon = coords
                ds_point = ds.sel(latitude=lat, longitude=lon, method="nearest")
                ds_point["degree_days"] = single_sine_horizontal_cutoff(
                    ds_point["tmin"], ds_point["tmax"], LTT, UTT
                )
                ds_point = ds_point.drop_vars(["tmin", "tmax"])
                ds_point.load()
                point_datasets.append(ds_point)
            elif isinstance(coords, dict) and all(k in coords for k in ["lat_min", "lat_max", "lon_min", "lon_max"]):
                # Bounding box case
                lat_min, lat_max = coords["lat_min"], coords["lat_max"]
                lon_min, lon_max = coords["lon_min"], coords["lon_max"]
                ds_bbox = ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
                ds_bbox["degree_days"] = single_sine_horizontal_cutoff(
                    ds_bbox["tmin"], ds_bbox["tmax"], LTT, UTT
                )
                ds_bbox = ds_bbox.drop_vars(["tmin", "tmax"])
                ds_bbox.load()
                point_datasets.append(ds_bbox)
            else:
                raise ValueError("Invalid coordinates format. Provide a tuple for a single point or a dictionary for a bounding box.")
            
            # Concatenate data for all points or bounding boxes along a new dimension 'point'
            ds_combined = xr.concat(point_datasets, dim="point")
            return ds_combined

    # Get a sorted list of file paths
    paths = sorted(glob(files))
    # Process each file and store the datasets in a list
    datasets = [process_one_path(p) for p in paths]
    # Concatenate all datasets along the specified dimension
    combined = xr.concat(datasets, "t")
    return combined

In [10]:
data = read_netcdfs( files=data_path + "PRISM/*/PRISM_temp_*.nc", coords = bbox_dict , LTT= LTT, UTT=UTT)

In [11]:
data

<xarray.Dataset> Size: 145kB
Dimensions:      (t: 9064, point: 1, longitude: 16, latitude: 0)
Coordinates:
  * t            (t) datetime64[ns] 73kB 2000-01-01 2000-01-02 ... 2024-10-24
  * longitude    (longitude) float64 128B -119.8 -119.8 -119.7 ... -119.2 -119.2
  * latitude     (latitude) float64 0B 
Dimensions without coordinates: point
Data variables:
    degree_days  (t, point) object 73kB None None None None ... None None None
Attributes:
    DODS_EXTRA.Unlimited_Dimension:  t